# NICT Badge & Wallet System — Code Examples

This notebook provides practical examples that reflect the workflows described in [`docs/workflows.md`](./workflows.md), including:
- User Registration
- Admin Registration
- User Information Update
- NFT Template Creation
- NFT Creation and Issuance to a User
- User Bingo Card Info Update
- Bingo Card Information Request

Before you start, make sure to set up your Python environment and initialize the database as described in the [Quick Start](../README.md#quick-start) section of README.


---
## Working with the Database

Most database operations use **SQLAlchemy ORM 2.0 style** sessions created via a `sessionmaker`. The pattern keeps transactions tidy and consistent.


### Step 1: Setup

Create an engine and sessionmaker:


In [ ]:
from nictbw.db.engine import make_engine
from sqlalchemy.orm import sessionmaker

engine = make_engine()  # This uses the DB_URL environment variable
Session = sessionmaker(engine)

### Step 2: Use the Session

Wrap your code inside a `with` block:


In [ ]:
with Session.begin() as session:
    # Your DB code goes here
    # ...
    pass  # Replace with real work in your application.

This pattern **automatically commits** if everything succeeds or **rolls back** if an exception is raised. 

You can also call `session.flush()` inside the block to write changes to the database before the block closes. This is useful when you need to access auto-generated fields (like primary keys) immediately after an insert.


---
## Notebook Helpers

The examples below generate unique identifiers so you can re-run cells without conflicting with previously inserted rows. Run the helper cell once before exploring the workflows.


In [ ]:
import uuid

EXAMPLE_SUFFIX = uuid.uuid4().hex[:8]
WORKFLOW_USER_IN_APP_ID = f"workflow_user_{EXAMPLE_SUFFIX}"
WORKFLOW_USER_SIGNUP_USERNAME = WORKFLOW_USER_IN_APP_ID
WORKFLOW_USER_SIGNUP_EMAIL = f"{WORKFLOW_USER_SIGNUP_USERNAME}@example.com"
WORKFLOW_USER_SIGNUP_PASSWORD = "ChangeMe123!"
WORKFLOW_USER_PAYMAIL = None
WORKFLOW_ADMIN_IN_APP_ID = f"workflow_admin_user_{EXAMPLE_SUFFIX}"
WORKFLOW_ADMIN_PAYMAIL = f"admin+{EXAMPLE_SUFFIX}@example.com"
WORKFLOW_TEMPLATE_PREFIX = f"WF-{EXAMPLE_SUFFIX}"
WORKFLOW_TEMPLATE_NAME = f"{EXAMPLE_SUFFIX}-Template"
WORKFLOW_TEMPLATE_SUBCATOGERY = f"{EXAMPLE_SUFFIX}-shop"
WORKFLOW_SHARED_KEY = f"shared-{EXAMPLE_SUFFIX}"

print("Using example suffix:", EXAMPLE_SUFFIX)
print("User in_app_id:", WORKFLOW_USER_IN_APP_ID)
print("Blockchain username:", WORKFLOW_USER_SIGNUP_USERNAME)
print("Blockchain email:", WORKFLOW_USER_SIGNUP_EMAIL)
print("User paymail (populated after registration):", WORKFLOW_USER_PAYMAIL)
print("Admin in_app_id:", WORKFLOW_ADMIN_IN_APP_ID)
print("Admin paymail:", WORKFLOW_ADMIN_PAYMAIL)
print("Template prefix:", WORKFLOW_TEMPLATE_PREFIX)
print("Shared key:", WORKFLOW_SHARED_KEY)


### Shared Imports

Load the models and workflows used throughout the notebook:


In [ ]:
from nictbw.models import Admin, NFTTemplate, User
from nictbw import workflows

---
## Workflow Examples

Each example below refers to a workflow described in [`docs/workflows.md`](./workflows.md).

Note that if you re-run cells, you might run into unique constraint violations (e.g., duplicate paymails). To solve this, simply re-run the helper cell above to generate a new unique suffix.


### Workflow: User Registration


In [ ]:
with Session.begin() as session:
    workflow_user = User(
        in_app_id=WORKFLOW_USER_IN_APP_ID,
        nickname="Workflow Explorer",
    )
    workflows.register_user(
        session,
        workflow_user,
        password=WORKFLOW_USER_SIGNUP_PASSWORD,
        email=WORKFLOW_USER_SIGNUP_EMAIL,
    )

    print(
        f"Registered user id={workflow_user.id} (paymail={workflow_user.paymail}, "
        f"on_chain_id={workflow_user.on_chain_id})"
    )

### Workflow: Admin Registration

In [ ]:
with Session.begin() as session:
    admin = Admin(
        paymail=WORKFLOW_ADMIN_PAYMAIL,
        password_hash="hashed-password-placeholder",
        name="Workflow Admin",
        role="event_manager",
    )
    session.add(admin)
    session.flush()  # Ensure admin.id is populated since we need it in the print statement
    print(f"Created admin id={admin.id}")

### Workflow: User Information Update

Load the user and adjust the attributes you need.

In [ ]:
with Session.begin() as session:
    user = User.get_by_in_app_id(session, WORKFLOW_USER_IN_APP_ID)
    if user is None:
        raise RuntimeError("Run the user registration example first.")

    user.nickname = "Workflow Explorer v2"
    user.set_password_hash("new-password-hash")
    session.flush()
    
    print(
        f"Updated user nickname to {user.nickname} (password hash set? {user.password_hash is not None})"
    )

### Workflow: NFT Template Creation


In [ ]:
with Session.begin() as session:
    admin = Admin.get_by_paymail(session, WORKFLOW_ADMIN_PAYMAIL)
    if admin is None:
        raise RuntimeError("Run the admin registration example first.")

    template = NFTTemplate(
        prefix=WORKFLOW_TEMPLATE_PREFIX,
        name=WORKFLOW_TEMPLATE_NAME,
        category="event",
        subcategory=WORKFLOW_TEMPLATE_SUBCATOGERY,
        created_by_admin_id=admin.id,
        description="Issued during the workflow walkthrough example.",
        triggers_bingo_card=True,
    )
    session.add(template)
    session.flush()
    print(
        f"Created template id={template.id} (triggers_bingo_card={template.triggers_bingo_card})"
    )

### Workflow: NFT Creation and Issuance to a User

**Note:** The following example involves **real** blockchain interaction. Therefore it requires:
- The `.env` file to be set up properly.
- The user to have a **valid paymail**, which is stored in `User.paymail` field.

If the paymail is not valid, the minting process will fail with an HTTP 400 Bad Request error.


In [ ]:
with Session.begin() as session:
    user = User.get_by_in_app_id(session, WORKFLOW_USER_IN_APP_ID)
    template = NFTTemplate.get_by_prefix(session, WORKFLOW_TEMPLATE_PREFIX)
    if user is None or template is None:
        raise RuntimeError("Ensure the user and template exist before minting.")

    nft = workflows.create_and_issue_nft(
        session=session,
        user=user,
        shared_key=WORKFLOW_SHARED_KEY,
        nft_template=template,
    )
    
    print(f"Minted NFT id={nft.id} with origin={nft.origin}")
    print(f"User now owns {len(user.nfts)} NFT(s).")

### Workflow: User Bingo Card Info Update

**Note:** Normally it is not necessary to call these methods directly, as the bingo card and cell info is automatically updated when the user gets a new NFT issued via the `create_and_issue_nft` workflow. This workflow is only needed when the bingo card or cell info gets out of sync for some reason.

In [ ]:
with Session.begin() as session:
    user = User.get_by_in_app_id(session, WORKFLOW_USER_IN_APP_ID)
    if user is None:
        raise RuntimeError("Run the user registration example first.")

    before_cards = len(user.bingo_cards)
    workflows.update_user_bingo_info(session, user)
    session.expire(user, ["bingo_cards"])
    after_cards = len(user.bingo_cards)

    print(f"Bingo cards before update: {before_cards}")
    print(f"Bingo cards after update: {after_cards}")

### Workflow: Bingo Card Information Request


You can get the user's bingo card information as a dictionary (JSON) or a JSON string, either in full or in a compact form.

In [ ]:
with Session.begin() as session:
    user = User.get_by_in_app_id(session, WORKFLOW_USER_IN_APP_ID)
    if user is None:
        raise RuntimeError("Run the user registration example first.")

    # full_payload = user.bingo_cards_json()
    # full_payload_str = json.dumps(full_payload, indent=2, ensure_ascii=False)
    compact_payload = user.bingo_cards_json(compact=True)
    compact_payload_str = user.bingo_cards_json_str(compact=True)
    print("Compact JSON string:")
    print(compact_payload_str)

In [ ]:
# For better readability in Jupyter notebooks
compact_payload